In [2]:
import pandas as pd
from scipy.interpolate import interp1d
from scipy.stats import norm
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import price_data as price
import ccxt


In [4]:
ftx = ccxt.ftx({
    'apiKey': 'mFRyLR4AAhLTc5RlWov3PKTcIbMHw3vGZwiHnsrn',
    'secret': 'oKaY1WEqTuhnNnq0iRi_Ry-CYckvE89-gPUPf21B',
    'enableRateLimit': True,
})

In [6]:
price.get_price_data('1d',symbol='ETH/USD')
price.get_price_data('1h',symbol='ETH/USD')

,unix,open,high,low,close
0,1620255600000,3523.9,3548.6,3498.1,3529.8
1,1620259200000,3529.8,3543.6,3452.6,3478.0
2,1620262800000,3477.6,3498.9,3443.4,3476.5
3,1620266400000,3476.5,3509.2,3464.8,3487.0
4,1620270000000,3487.0,3487.0,3451.3,3474.0
...,...,...,...,...,...
1496,1625641200000,2384.7,2410.4,2362.1,2379.3
1497,1625644800000,2379.3,2400.7,2373.7,2383.5
1498,1625648400000,2383.5,2390.8,2366.7,2379.6
1499,1625652000000,2379.6,2382.9,2364.3,2373.6


In [8]:
ftx.has['createMarketOrder']

True